In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!java --version

openjdk 11.0.9.1 2020-11-04
OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [ ]:
cd drive/MyDrive/ML_German/code

/content/drive/MyDrive/ML_German/code


In [ ]:
!javac BlendedSpectrumStringKernel.java

Note: BlendedSpectrumStringKernel.java uses unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


In [ ]:
!java -Xmx5096m ComputeStringKernel 1 spectrum 3 3 ./texts/texts.txt ./kernels/my_kern.txt 

Loaded 22583 samples from ./texts/texts.txt
Computing the spectrum kernel based on 3-3-grams ...
Computed kernel to row 99 in 53840 ms
Computed kernel to row 199 in 52686 ms
Computed kernel to row 299 in 53254 ms
Computed kernel to row 399 in 52547 ms
Computed kernel to row 499 in 53063 ms
Computed kernel to row 599 in 52341 ms
Computed kernel to row 699 in 53405 ms
Computed kernel to row 799 in 52703 ms
Computed kernel to row 899 in 51002 ms
Computed kernel to row 999 in 51900 ms
Computed kernel to row 1099 in 53374 ms
Computed kernel to row 1199 in 51857 ms
Computed kernel to row 1299 in 49552 ms
Computed kernel to row 1399 in 51260 ms
Computed kernel to row 1499 in 50631 ms
Computed kernel to row 1599 in 51159 ms
Computed kernel to row 1699 in 49920 ms
Computed kernel to row 1799 in 49226 ms
Computed kernel to row 1899 in 48703 ms
Computed kernel to row 1999 in 50956 ms
Computed kernel to row 2099 in 49337 ms
Computed kernel to row 2199 in 49495 ms
Computed kernel to row 2299 in 469

In [ ]:
!javac BlendedSpectrumStringKernel.java

Note: BlendedSpectrumStringKernel.java uses unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


In [ ]:
!javac ComputeStringKernel.java

In [ ]:
!java -Xmx5096m ComputeStringKernel 2 spectrum 3 3 ./texts/valid_texts.txt ./texts/texts.txt ./kernels/valid_kern1.txt 

Loaded 3044 samples from ./texts/valid_texts.txt
Loaded 22583 samples from ./texts/texts.txt
Computing the spectrum kernel based on 3-3-grams ...
Computed kernel to row 99 in 50771 ms
Computed kernel to row 199 in 49798 ms
Computed kernel to row 299 in 50773 ms
Computed kernel to row 399 in 50272 ms
Computed kernel to row 499 in 51017 ms
Computed kernel to row 599 in 50287 ms
Computed kernel to row 699 in 50647 ms
Computed kernel to row 799 in 49901 ms
Computed kernel to row 899 in 49225 ms
Computed kernel to row 999 in 49790 ms
Computed kernel to row 1099 in 50153 ms
Computed kernel to row 1199 in 49997 ms
Computed kernel to row 1299 in 50793 ms
Computed kernel to row 1399 in 50874 ms
Computed kernel to row 1499 in 51601 ms
Computed kernel to row 1599 in 52747 ms
Computed kernel to row 1699 in 50496 ms
Computed kernel to row 1799 in 50230 ms
Computed kernel to row 1899 in 51559 ms
Computed kernel to row 1999 in 49155 ms
Computed kernel to row 2099 in 50181 ms
Computed kernel to row 21

In [5]:
import nltk
import pandas as pd
import numpy as np
import copy
from typing import List
import re
from sklearn.model_selection import KFold 
from sklearn.svm import SVR, NuSVR
import joblib

In [2]:
cd drive/MyDrive/ML_German

/content/drive/MyDrive/ML_German


In [ ]:
# def parse_data(file_in: str, file_out: str) -> str:
#   lines = nltk.load(file_in).split('\n');
#   ids, latitudes, longitudes, texts = [], [], [], []
#   for line in lines[:-1]:
#     splits = line.split(',');
#     ids.append(splits[0]);
#     latitudes.append(splits[1]);
#     longitudes.append(splits[2]);
#     texts.append(" ".join(splits[3:]));
#   date = pd.DataFrame(list(zip(ids, latitudes, longitudes, texts)), columns=['Id', 'Lat', 'Long', 'Text'])
#   date.to_csv(file_out, index = False)
#   return file_out;  

In [9]:
def parse_data(file_in: str, file_out: str) -> str:
  lines = nltk.load(file_in).split('\n');
  ids, texts = [], []
  for line in lines[:-1]:
    splits = line.split(',');
    ids.append(splits[0]);
    texts.append(" ".join(splits[1:]));
  date = pd.DataFrame(list(zip(ids, texts)), columns=['Id', 'Text'])
  date.to_csv(file_out, index = False)
  return file_out;  

In [10]:
test_data_file = parse_data('./data_files/test.txt', './csv_files/test.csv');

In [ ]:
# training_data_file = parse_data('training.txt', 'training.csv');
# validation_data_file = parse_data('validation.txt', 'validation.csv');

In [ ]:
cd code

/content/drive/My Drive/ML_German/code


In [ ]:
validation_data = pd.read_csv('../csv_files/validation.csv')
validation_data.head()

,Id,Lat,Long,Text
0,203001,52.098389,10.354496,"""wenn mer anere Party bi Kollege en neue Bro f..."
1,200313,51.629866,7.723453,"""Heii guetä wuchestart gha? Jo wunderbar und d..."
2,201966,52.242550,8.967590,"""Wieso ned? De werds eim emel ned langwiilig. ..."
3,201123,52.242550,8.967590,"""Hani welle vo zueri uf lausanne denn hetts e ..."
4,200374,51.701946,8.416906,"""Mir isch die Applikation plagiert worde as Di..."


In [ ]:
valid_texts = [text.lower() for text in validation_data['Text']]
with open('valid_texts.txt', "w") as file:
  for text in valid_texts:
    file.write(text + "\n")

In [ ]:
valid_latitudes = validation_data['Lat'].to_numpy()
valid_latitudes

array([52.09838926, 51.62986577, 52.24255034, ..., 51.88486301,
       51.77452055, 52.28945205])

In [ ]:
with open('./kernels/valid_kern.txt', "r") as file:
  mat = [[int(num) for num in line.split(' ') if num != '\n'] for line in file]
valid_kernel = np.array(mat)
valid_kernel

array([[ 23, 121,  41, ...,  22, 353,  38],
       [ 54, 253,  65, ...,  41, 678,  63],
       [ 20,  74,   2, ...,  17, 193,  19],
       ...,
       [ 39, 171,  34, ...,  29, 466,  55],
       [ 57, 192,  57, ...,  44, 571,  31],
       [ 43, 196,  41, ...,  27, 640,  57]])

In [ ]:
np.savez_compressed('valid_kernel.npz', valid_kernel)

In [ ]:
training_data = pd.read_csv('../csv_files/training.csv')
training_data.head()

,Id,Lat,Long,Text
0,119165,51.810067,10.191331,"""Seit d Vase: """"Wenn ich kaputt gang bringt d..."
1,100377,51.918188,10.599245,"""Haha bin au w isch der amig au so richtig lan..."
2,109550,52.711074,9.987374,"""isch d hiltl dachterrasse amne samstig viel b..."
3,111440,52.386711,11.700612,"""Ich fühle mich wie die Weimarer Republik... ...."
4,116670,52.314631,9.701835,"""Eui liebschte Lunchidee zum Mitneh? 😬 En Gril..."


In [ ]:
k = 5
kf = KFold(n_splits=k, random_state=42, shuffle=True)

In [ ]:
t = 0
for train_index, test_index in kf.split(training_data):
    X_train , X_test = training_data.iloc[train_index,:], training_data.iloc[test_index,:]
    #y_train , y_test = y[train_index] , y[test_index]
    with open('texts' + str(t) + '.txt', "w") as file:
      for text in X_train['Text']:
        file.write(text + "\n")
    t += 1

In [ ]:
texts = [text.lower() for text in training_data['Text']]

In [ ]:
latitudes = training_data['Lat'].to_numpy()
latitudes

array([51.81006711, 51.91818792, 52.71107383, ..., 50.59753425,
       51.84808219, 52.95150685])

In [ ]:
with open('./kernels/my_kern.txt', "r") as file:
  mat = [[int(num) for num in line.split(' ') if num != '\n'] for line in file]
training_kernel = np.array(mat)
training_kernel

In [ ]:
np.savez_compressed('my_kernel.npz', kernel)

In [ ]:
dict_data = np.load('./kernels/training_kernel.npz')
training_kernel = dict_data['arr_0']
training_kernel

array([[ 620,  313,   97, ...,  190,  228,  314],
       [ 313, 1348,  208, ...,  451,  596,  566],
       [  97,  208,  286, ...,  147,  160,  169],
       ...,
       [ 190,  451,  147, ..., 1045,  425,  377],
       [ 228,  596,  160, ...,  425, 1185,  450],
       [ 314,  566,  169, ...,  377,  450, 1366]])

In [ ]:
svr = SVR(kernel='precomputed', verbose=True)

In [ ]:
svr.fit(kernel, latitudes)

[LibSVM]

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='precomputed', max_iter=-1, shrinking=True, tol=0.001, verbose=True)

In [ ]:
from sklearn.externals import joblib
joblib_file = "joblib_model.pkl"
joblib.dump(svr, joblib_file)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['joblib_model.pkl']

In [ ]:
joblib.dump(svr, 'svr.pkl')

['svr.pkl']

In [ ]:
svr = joblib.load('svr.pkl')

In [ ]:
with open('./kernels/valid_kern1.txt', "r") as file:
  mat = [[int(num) for num in line.split(' ') if num != '\n'] for line in file]
kernel1 = np.array(mat)
kernel1

array([[ 23,  54,  20, ...,  39,  57,  43],
       [121, 253,  74, ..., 171, 192, 196],
       [ 41,  65,   2, ...,  34,  57,  41],
       ...,
       [ 22,  41,  17, ...,  29,  44,  27],
       [353, 678, 193, ..., 466, 571, 640],
       [ 38,  63,  19, ...,  55,  31,  57]])

In [ ]:
np.array_equal(kernel1, np.transpose(valid_kernel))

True

In [ ]:
pred_valid_lats = svr.predict(kernel1)

In [ ]:
mae = 0
for v_lat, pred_lat in zip(valid_latitudes, pred_valid_lats):
  mae += abs(v_lat - pred_lat)
print(mae/len(valid_latitudes))

1.012048622569786


In [ ]:
nuSvr = NuSVR(kernel='precomputed', C = 10, cache_size = 2048, verbose=True)

17:53

In [ ]:
nuSvr.fit(training_kernel, latitudes)

[LibSVM]

In [ ]:
joblib.dump(nuSvr, 'svr.pkl')

In [ ]:
!java -Xmx5096m ComputeStringKernel 1 spectrum 3 5 ./texts/train_text3.txt ./kernels/train_kern3.txt 

Loaded 16938 samples from ./texts/train_text3.txt
Computing the spectrum kernel based on 3-5-grams ...
Computed kernel to row 99 in 112555 ms
Computed kernel to row 199 in 108743 ms
Computed kernel to row 299 in 116986 ms
Computed kernel to row 399 in 114385 ms
Computed kernel to row 499 in 116674 ms
Computed kernel to row 599 in 115366 ms
Computed kernel to row 699 in 111923 ms
Computed kernel to row 799 in 111221 ms
Computed kernel to row 899 in 114114 ms
Computed kernel to row 999 in 109499 ms
Computed kernel to row 1099 in 109970 ms
Computed kernel to row 1199 in 114486 ms
Computed kernel to row 1299 in 110605 ms
Computed kernel to row 1399 in 107453 ms
Computed kernel to row 1499 in 113565 ms
Computed kernel to row 1599 in 99942 ms
Computed kernel to row 1699 in 99986 ms
Computed kernel to row 1799 in 98293 ms
Computed kernel to row 1899 in 100859 ms
Computed kernel to row 1999 in 100800 ms
Computed kernel to row 2099 in 97328 ms
Computed kernel to row 2199 in 102034 ms
Computed k

In [ ]:
!java -Xmx5096m ComputeStringKernel 2 spectrum 3 5 ./texts/train_text1.txt ./texts/test_text1.txt ./kernels/test_kern1.txt 

Loaded 16937 samples from ./texts/train_text1.txt
Loaded 5646 samples from ./texts/test_text1.txt
Computing the spectrum kernel based on 3-5-grams ...
Computed kernel to row 99 in 40337 ms
Computed kernel to row 199 in 37017 ms
Computed kernel to row 299 in 40230 ms
Computed kernel to row 399 in 37535 ms
Computed kernel to row 499 in 41325 ms
Computed kernel to row 599 in 40288 ms
Computed kernel to row 699 in 38753 ms
Computed kernel to row 799 in 40009 ms
Computed kernel to row 899 in 39670 ms
Computed kernel to row 999 in 38698 ms
Computed kernel to row 1099 in 39420 ms
Computed kernel to row 1199 in 42249 ms
Computed kernel to row 1299 in 39263 ms
Computed kernel to row 1399 in 38388 ms
Computed kernel to row 1499 in 40065 ms
Computed kernel to row 1599 in 39762 ms
Computed kernel to row 1699 in 39102 ms
Computed kernel to row 1799 in 39397 ms
Computed kernel to row 1899 in 40081 ms
Computed kernel to row 1999 in 40078 ms
Computed kernel to row 2099 in 40003 ms
Computed kernel to r

In [ ]:
train_no_emoji = pd.read_csv('../csv_files/training_no_emoji.csv')

In [ ]:
with open('./texts/training_no_emoji.txt', "w") as file:
  for text in train_no_emoji['Text']:
    file.write(text.lower() + "\n")

In [ ]:
!java -Xmx5096m ComputeStringKernel 1 spectrum 3 5 ./texts/training_no_emoji.txt ./kernels/train_no_emoji_kernel.txt

Loaded 22583 samples from ./texts/training_no_emoji.txt
Computing the spectrum kernel based on 3-5-grams ...
Computed kernel to row 99 in 170203 ms
Computed kernel to row 199 in 163186 ms
Computed kernel to row 299 in 169683 ms
Computed kernel to row 399 in 165142 ms
Computed kernel to row 499 in 166960 ms
Computed kernel to row 599 in 165422 ms
Computed kernel to row 699 in 170628 ms
Computed kernel to row 799 in 168516 ms
Computed kernel to row 899 in 160613 ms
Computed kernel to row 999 in 156072 ms
Computed kernel to row 1099 in 169840 ms
Computed kernel to row 1199 in 165302 ms
Computed kernel to row 1299 in 154654 ms
Computed kernel to row 1399 in 162878 ms
Computed kernel to row 1499 in 160447 ms
Computed kernel to row 1599 in 162326 ms
Computed kernel to row 1699 in 156938 ms
Computed kernel to row 1799 in 155209 ms
Computed kernel to row 1899 in 153587 ms
Computed kernel to row 1999 in 162657 ms
Computed kernel to row 2099 in 152494 ms
Computed kernel to row 2199 in 149714 ms
